In [1]:
import numpy as np
import pandas as pd
from keras.layers import Embedding
import scipy.sparse

from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import Conv1D, MaxPooling1D
from keras.datasets import imdb

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


#### Define Data Functions

In [2]:
def getData():
    train = pd.read_csv('../output/train_pre.csv')
    test = pd.read_csv('../output/test_pre.csv')
    return train,test

def getFeature(Name,CharFlag=False):
    
    if CharFlag:
        train_features = scipy.sparse.load_npz('../output/train_features.npz')
        test_features = scipy.sparse.load_npz('../output/test_features.npz')
        return train_featuress.todense(),test_features.todense()
    else :
        if name == 'fastEeb':
            fastEeb = np.load('../output/embedMatrix.npy',encoding='latin1')
            return fastEeb
        else : 
            globeEbd = np.load('../output/embedMatrixglove.npy',encoding='latin1')
            return globeEbd

def getSequence():
    train_sequence = np.load('../output/x_train_seq.npy',encoding='latin1')
    test_sequence = np.load('../output/x_test_seq.npy',encoding='latin1')
    return train_sequence,test_sequence

def getIP(train,test):
    train_ip = train[['ip_0','ip_1', 'ip_2', 'ip_3']]
    train_ip['index'] = 1
    test_ip = test[['ip_0','ip_1', 'ip_2', 'ip_3']]
    test_ip['index'] = 0
    result = pd.concat([train_ip,test_ip])

    result = pd.get_dummies(result, prefix=['ip_0','ip_1', 'ip_2', 'ip_3'], columns=['ip_0','ip_1', 'ip_2', 'ip_3'],drop_first=True)
    train_ip = result[result['index']==1]
    test_ip = result[result['index']==0]
    return train_ip,test_ip

#### Usage

``` python
train,test = getData()
train_features,test_features = getFeature('',CharFlag=True)
train_sequence,test_sequence = getSequence()
train_ip,test_ip = getIP(train,test)
```

#### Define Columns

In [ ]:
xCol = ['ip_0','ip_1', 'ip_2', 'ip_3','caps_vs_length',
          'num_punctuation', 'num_unique_words','num_exclamation_marks']
yCol = ['toxic','severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
allCol = xCol+yCol

In [21]:
x_train = train[xCol][:1000]
y_train = train[yCol][:1000]

X_train, X_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.1, random_state=42)

In [149]:
x_train = train_sequence
y_train = train[yCol]

X_train, X_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.1, random_state=42)

In [150]:
def model_fit(X_train, X_test, y_train, y_test):
    n_x,n_m = X_train.shape
    n_y = y_test.shape[1]
    model = Sequential()
    model.add(Dense(n_x, activation='relu', input_dim=n_m))
    model.add(Dense(n_y, activation='softmax'))
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=3, batch_size=256)
    model.evaluate(X_test,y_test,batch_size=256)

In [ ]:
def LSTM01():
    # Embedding
    max_features = 20000
    maxlen = 150
    embedding_size = 128

    # Convolution
    kernel_size = 5
    filters = 64
    pool_size = 4

    # LSTM
    lstm_output_size = 70

    # Training
    batch_size = 128
    epochs = 8
    model = Sequential()
    model.add(Embedding(max_features, embedding_size, input_length=maxlen))
    model.add(Dropout(0.25))
    model.add(Conv1D(filters,
                     kernel_size,
                     padding='valid',
                     activation='relu',
                     strides=1))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(LSTM(lstm_output_size))
    model.add(Dense(6))
    model.add(Activation('sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    print('Train...')
    model.fit(X_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(X_test, y_test))
    score, acc = model.evaluate(X_test, y_test, batch_size=batch_size)
    print('Test score:', score)
    print('Test accuracy:', acc)
    model.save_weights("LSTM01.h5")
    predict(LSTM01,model,)

In [169]:
def predict(name,model,testSet):
    y_pred = model.predict(testSet, batch_size=1024, verbose=1)
    submission = pd.read_csv('../input/sample_submission.csv')
    submission[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]] = y_pred
    submission.to_csv('LSTM01.csv', index=False)

In [ ]:
y_pred = model.predict(x_test, batch_size=1024, verbose=1)

submission = pd.read_csv('../input/sample_submission.csv')
submission[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]] = y_pred
submission.to_csv('submission.csv', index=False)
model.save_weights('best.hdf5')
model.save_weights("model")